# Putting Together Webscraping HTMLs and parsing through the page
by Harsha and Jesse

In [1]:
# BeautifulSoup and requests library import
from bs4 import BeautifulSoup
import requests 
import re
import pandas as pd

In [8]:
#cleaned htmls (entire html should work)
htmls = pd.read_csv("cleaned_htmls.csv", index_col = 0, header = None).reset_index(drop=True)

In [31]:
#needs http before else will not work
htmls = "http://" + htmls

In [32]:
#create a list from the dataframe with htmls
h = htmls[1].to_list()

In [40]:
#create a dictionary to fill with keys: urls, data: data from article
htmls_with_data = {}

htmls_with_data

after 50 articles the 3 second sleep ran into an error so using 5 second sleep instead.

In [ ]:
#making for loop to scrape the data:
counter = 1

for i in h[53:]:
    time.sleep(5)
    #page request
    page_response = requests.get(i, timeout=5)
    print(page_response)
    
    #parsing content from page and storing it
    page_content = BeautifulSoup(page_response.content, "html.parser")
    
    #empty list to store user data
    data = []
    
    try:
        #grabs div that stores title info
        titleInfo = page_content.find_all('div',attrs={"class":"n p"})

        #finds title within titleInfo div
        href = str(titleInfo).index("<h1")
        start = str(titleInfo)[href:].index(">")+href+1
        end = str(titleInfo)[href:].index("</h1>")+href

        #appends title to data list after removing extra tags
        title = re.sub(re.compile('<.*?>'), '', str(titleInfo)[start:end])
        data.append(title)

        #<h1>Finding and Adding Author to Data</h1>

        #grabs div that stores user info
        userInfo = page_content.find_all('div',attrs={"class":"o n"})

        #finds authors name within userInfo div
        href = str(userInfo)[24:].index("<a class=")+24
        start = str(userInfo)[href:].index(">")+href+1
        end = str(userInfo)[href:].index("</a>")+href

        #appends authors name to data list
        author = str(userInfo)[start:end]
        data.append(author)

        #<h1>Finding and Adding Author Page URL to Data</h1>

        #finds date within userInfo div
        href = str(userInfo)[24:].index("<a class=")
        start = str(userInfo)[href:].index("href=")+href+6
        end = str(userInfo)[start:].index("?source")+start

        #appends date to data list
        url = "medium.com" + str(userInfo)[start:end]
        data.append(url)

        #<h1>Finding and Adding Date to Data</h1>

        #finds date within userInfo div
        href = str(userInfo)[end:].index("<a class=")+end
        start = str(userInfo)[href:].index(">")+href+1
        end = str(userInfo)[href:].index("</a>")+href

        #appends date to data list
        date = str(userInfo)[start:end]
        data.append(date)

        #<h1>Finding and Adding Number of Claps to Data</h1>

        #grabs all paragraphs in the article
        clapInfo = page_content.find_all('div',attrs={"class":"n o"})

        #finds claps within clapInfo div
        result = re.sub(re.compile('<.*?>'), '', str(clapInfo))
        result = re.findall("\d+", result)

        #appends claps to data list
        if len(result) == 0:
            data.append('0')

        else:
            data.append(result[0])

        #<h1>Adding a List of the Text Content to Data</h1>

        #grabs all paragraphs in the article
        textContent = page_content.find_all("p")
        textInfo = str(textContent)

        #RegEx that cleans paragraph data by removing html tags and extra commas
        result = re.sub(re.compile('<.*?>'), '', textInfo)
        result = re.split("\., ", result)
        result = result[:-1]
        data.append(result)
    
        #append html to dictionary with data
        htmls_with_data[i] = data
    except:
        htmls_with_data[i] = None
    
    #counter
    print(counter)
    counter = counter + 1

In [63]:
#turn dictionary into a df
dictionary_of_data = pd.DataFrame(htmls_with_data)

In [71]:
#pivot to make the index the url and resetting to get it
full_data = dictionary_of_data.T.reset_index()

In [73]:
#change the urls to be something easier to interpret
full_data.columns = ["url", "title", "author", "username", "published", "claps", "text"]

In [75]:
#to csv
full_data.to_csv("Medium_Articles_Art_Data.csv")